About: [Wikipedia](https://en.wikipedia.org/wiki/Brainfuck)

   - "Words" are unsigned 8-bit, so 0...255 inclusive.
   - Attempt to read past end of input leaves cell unchanged-- seems to be original behaviour.


In [147]:
class DataBuffer:
    """DataBuffer: a class to hold BF data.  Essentially implements a dictionary
    with an integer lookup, but optimised for contiguous data."""
    def __init__(self):
        self.length = 1
        self.buffer = [0] * self.length
        self.offset = 0
    
    def _rebase(self, newOffset):
        """Internal: change the offset to `newOffset`."""
        if newOffset > offset:
            raise Exception("In this implementation, `newOffset` should always be smaller than"
                            "current offset")
        self.buffer = ( [0] * (self.offset - newOffset) ) + self.buffer
        self.offset = newOffset
        self.length += self.offset - newOffset
    
    def _grow(self, newLength):
        """Internal: extend the length of the buffer."""
        if newLength < self.length:
            raise Exception("In this implementation, `newLength` should be larger than old length")
        self.buffer = self.buffer + ( [0] * (newLength - self.length) )
        self.length = newLength
        
    def _give_access(self, index):
        """Internal: Allow `index` to refer to a valid place in the buffer."""
        # TODO: Make this grow intelligently
        if index < self.offset:
            self._rebase(index - 16)
        if index - self.offset >= self.length:
            self._grow(index - self.offset + 16)
        
    def __getitem__(self, index):
        if index < self.offset or index - self.offset >= self.length:
            return 0
        return self.buffer[index - self.offset]
    
    def __setitem__(self, index, value):
        self._give_access(index)
        self.buffer[index - self.offset] = value
        
    def __repr__(self):
        return "DataBuffer(offset={}, length={})".format(self.offset, self.length)
    
    def __str__(self):
        ashex = ( hex(x)[2:] for x in self.buffer )
        ashex = ( x if len(x) == 2 else "0"+x for x in ashex )        
        return " ".join( ashex )

In [148]:
class BFInterpretter:
    def __init__(self, script, bytesin, strict = True):
        self.buffer = DataBuffer()
        self.inPos = 0
        self.dataPos = 0
        self.output = []
        self.bytesin = bytesin[:]
        self.mode = strict
        # Strip whitespace
        self.script = ""
        temp = script[:]
        while len(temp) > 0:
            temp = temp.lstrip()
            self.script += temp[0]
            temp = temp[1:]
        
    def _skip_to_matching_close(self):
        if self.script[self.inPos] != "[":
            raise Exception("Should call with instruction pointer on `[`")
        bracket_count = 1
        self.inPos += 1
        while bracket_count != 0:
            if self.script[self.inPos] == "[":
                bracket_count += 1
            elif self.script[self.inPos] == "]":
                bracket_count -= 1
            self.inPos += 1

    def _skip_to_matching_open(self):
        if self.script[self.inPos] != "]":
            raise Exception("Should call with instruction pointer on `]`")
        bracket_count = 1
        while bracket_count != 0:
            self.inPos -= 1
            if self.script[self.inPos] == "[":
                bracket_count -= 1
            elif self.script[self.inPos] == "]":
                bracket_count += 1
        
    def step(self):
        """Perform one instruction."""
        if self.inPos >= len(self.script): return
        if self.script[self.inPos] == ">":
            self.dataPos += 1
        elif self.script[self.inPos] == "<":
            self.dataPos -= 1
        elif self.script[self.inPos] == "+":
            self.buffer[self.dataPos] = (self.buffer[self.dataPos] + 1) % 256
        elif self.script[self.inPos] == "-":
            self.buffer[self.dataPos] = (self.buffer[self.dataPos] - 1) % 256
        elif self.script[self.inPos] == ".":
            self.output.append(self.buffer[self.dataPos])
        elif self.script[self.inPos] == ",":
            if len(self.bytesin) > 0:
                self.buffer[self.dataPos] = ord(self.bytesin[0]) % 256
                self.bytesin = self.bytesin[1:]
        else:
            # Other commands
            if self.script[self.inPos] == "[":
                if self.buffer[self.dataPos] != 0:
                    self.inPos += 1
                else:
                    self._skip_to_matching_close()
            elif self.script[self.inPos] == "]":
                self._skip_to_matching_open()
            else:
                if self.mode:
                    raise Exception("Unknown command: '{}'".format(self.script[self.inPos]))
                else:
                    self.inPos += 1 # Skip unknown
            return
        self.inPos += 1
            
    def run(self):
        while self.inPos < len(self.script):
            self.step()
        return self.output
    
    def __repr__(self):
        ret = "BFInterpretter: Instr {} / {}\n".format(self.inPos, len(self.script))
        return ret + "Current output: '{}'".format("".join(chr(x) for x in self.output))
    
    def as_string(self):
        return "".join(chr(x) for x in self.output)
    
    def print_machine(self):
        """Prints out a representation of the machine, with a window of 15 characters each side."""
        """Also prints out ascii, defined here as numbers from 32 to 126 inclusive."""
        start = self.dataPos - 15
        end = self.dataPos + 15
        hexline = ( hex( self.buffer[i] )[2:] for i in range(start, end+1) )
        hexline = ( "0" + x if len(x) == 1 else x for x in hexline )
        asciiline = ( " " + chr( self.buffer[i] ) if self.buffer[i]>=32 and self.buffer[i]<=126 else "  "
                     for i in range(start, end+1) )
        start = self.inPos - 30
        end = self.inPos + 30
        cmdline = ( " " if i < 0 or i >= len(self.script) else self.script[i] for i in range(start, end+1) )
        return ( " ".join(hexline) + "\n" + " ".join(asciiline) + "\n"
                + " "*46 + "^\n"
                + "".join(cmdline) + "\n"
                + " "*30 + "^\n"
                + "Output: '{}'".format(self.as_string()) )

In [149]:
bf = BFInterpretter("++++++++[>++++[>++>+++>+++>+<<<<-]>+>+>->>+[<]<-]>>.>---.+++++++..+++.>>.<-.<.+++.------.--------.>>+.>++."
                    , "")
bf.run()

[72, 101, 108, 108, 111, 32, 87, 111, 114, 108, 100, 33, 10]

In [150]:
bf

BFInterpretter: Instr 106 / 106
Current output: 'Hello World!
'

In [151]:
bf.as_string()

'Hello World!\n'

In [131]:
str(bf.buffer)

'00 00 48 64 57 21 0a 00 00 00 00 00 00 00 00 00 00'

## Visual working machine ##

In [132]:
bf = BFInterpretter("++++++++[>++++[>++>+++>+++>+<<<<-]>+>+>->>+[<]<-]>>.>---.+++++++..+++.>>.<-.<.+++.------.--------.>>+.>++."
                    , "")
print(bf.print_machine())

00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00
                                                                                            
                                              ^
                              ++++++++[>++++[>++>+++>+++>+<<<
                              ^
Output: ''


In [133]:
for _ in range(1000):
    bf.step()
    print(bf.print_machine())

00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 01 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00
                                                                                            
                                              ^
                             ++++++++[>++++[>++>+++>+++>+<<<<
                              ^
Output: ''
00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 02 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00
                                                                                            
                                              ^
                            ++++++++[>++++[>++>+++>+++>+<<<<-
                              ^
Output: ''
00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 03 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00
                                                                                            
                                              ^
                           ++++++++[>++++[>++>+++>+++>+<<<<-]
                          

## Various Tests ##

From http://esoteric.sange.fi/brainfuck/bf-source/prog/ file `tests.b`

In [134]:
bf = BFInterpretter(">,>+++++++++,>+++++++++++[<++++++<++++++<+>>>-]<<.>.<<-.>.>.<<.", "\n")
bf.run()
bf

BFInterpretter: Instr 63 / 63
Current output: 'LK
LK
'

In [140]:
bf = BFInterpretter("++++[>++++++<-]>[>+++++>+++++++<<-]>>++++<[[>[[>>+<<-]<]>>>-]>-[>+>+<<-]>]+++++[>+++++++<<++>-]>.<<.", "")
bf.run()
bf

BFInterpretter: Instr 100 / 100
Current output: '#
'

In [141]:
bf.as_string()

'#\n'

In [143]:
bf = BFInterpretter("[]++++++++++[>++++++++++++++++++>+++++++>+<<<-]A;?@![#>>+<<]>[>++<[-]]>.>.", "", False)
bf.run()

[72, 10]

In [146]:
bf.as_string() == "H\n"

True

## ROT13 example ##

In [123]:
bf = BFInterpretter(""",
[>+<-[>+<-[>+<-[>+<-[>+<-[>+<-[>+<-[>+<-[>+<-[>+<-[>+<-[>+<-[>+<-[>+<-[>+<-[>+<-
[>+<-[>+<-[>+<-[>+<-[>+<-[>+<-[>+<-[>+<-[>+<-[>+<-[>+<-[>+<-[>+<-[>+<-[>+<-[>+<-
[>+<-[>+<-[>+<-[>+<-[>+<-[>+<-[>+<-[>+<-[>+<-[>+<-[>+<-[>+<-[>+<-[>+<-[>+<-[>+<-
[>+<-[>+<-[>+<-[>+<-[>+<-[>+<-[>+<-[>+<-[>+<-[>+<-[>+<-[>+<-[>+<-[>+<-[>+<-[>+<-
[>++++++++++++++<-
[>+<-[>+<-[>+<-[>+<-[>+<-[>+<-[>+<-[>+<-[>+<-[>+<-[>+<-[>+<-
[>>+++++[<----->-]<<-
[>+<-[>+<-[>+<-[>+<-[>+<-[>+<-[>+<-[>+<-[>+<-[>+<-[>+<-[>+<-
[>++++++++++++++<-
[>+<-[>+<-[>+<-[>+<-[>+<-
[>++++++++++++++<-
[>+<-[>+<-[>+<-[>+<-[>+<-[>+<-[>+<-[>+<-[>+<-[>+<-[>+<-[>+<-
[>>+++++[<----->-]<<-
[>+<-[>+<-[>+<-[>+<-[>+<-[>+<-[>+<-[>+<-[>+<-[>+<-[>+<-[>+<-
[>++++++++++++++<-
[>+<-]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]
]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]>.[-]<,]""",
                    "~mlk zyx")
bf.run()

[126, 122, 121, 120, 32, 109, 108, 107]

In [126]:
bf.as_string() == "~zyx mlk"

True